In [7]:
import pandas as pd
from google_play_scraper import reviews
from datetime import datetime

In [ ]:
# --- Pengaturan ---
app_id = 'com.tokopedia.tkpd'

# Tentukan rentang waktu yang Anda inginkan (termasuk jam dan menit jika perlu)
start_date = datetime(2024, 2, 4)
end_date = datetime(2025, 9, 29)

all_reviews_list = []
continuation_token = None
# ------------------

print(f"Memulai scraping untuk ulasan dari {start_date} hingga {end_date}...")

# Loop akan terus berjalan sampai kita menghentikannya secara manual
while True:
    result, continuation_token = reviews(
        app_id,
        country='id',
        count=200,  # Ambil 200 ulasan per halaman
        continuation_token=continuation_token
    )

    # Tambahkan hasil ulasan yang baru didapat ke list utama
    all_reviews_list.extend(result)
    print(f"Total ulasan terkumpul: {len(all_reviews_list)}")
    
    # Cek jika tidak ada hasil lagi atau tidak ada token berikutnya
    if not continuation_token or not result:
        print("Tidak ada ulasan lebih lanjut. Proses scraping berhenti.")
        break
        
    # Ambil tanggal dari ulasan TERAKHIR di batch saat ini
    last_review_date = result[-1]['at']

    # Jika tanggal ulasan terakhir sudah lebih tua dari start_date kita,
    # kita bisa berhenti karena halaman-halaman berikutnya pasti lebih tua lagi.
    if last_review_date < start_date:
        print("Sudah mencapai batas tanggal mulai. Proses scraping berhenti.")
        break

# --- Proses Filtering (Dengan Perbaikan) ---
print("\nMemulai proses filtering data...")

df = pd.DataFrame(all_reviews_list)

# 1. Konversi kolom 'at' menjadi format datetime
df['at'] = pd.to_datetime(df['at'])

# 2. HAPUS INFORMASI TIMEZONE UNTUK MENSTANDARKAN (INI PERBAIKANNYA)
df['at'] = df['at'].dt.tz_localize(None)

# 3. Lakukan filtering akhir yang sekarang sudah akurat
filtered_df = df[
    (df['at'] >= start_date) & (df['at'] <= end_date)
]

print(f"Jumlah ulasan setelah difilter secara akurat: {len(filtered_df)}")

# Simpan hasil akhir ke CSV
if not filtered_df.empty:
    file_name = f'tokopedia_reviews_FIXED_{start_date.date()}_to_{end_date.date()}.csv'
    filtered_df.to_csv(file_name, index=False)
    print(f"File '{file_name}' berhasil disimpan.")
else:
    print("Tidak ada ulasan yang ditemukan pada rentang tanggal tersebut.")

Memulai scraping untuk ulasan dari 2024-02-04 00:00:00 hingga 2025-09-29 00:00:00...
Total ulasan terkumpul: 200
Total ulasan terkumpul: 400
Total ulasan terkumpul: 600
Total ulasan terkumpul: 800
Total ulasan terkumpul: 1000
Total ulasan terkumpul: 1200
Total ulasan terkumpul: 1400
Total ulasan terkumpul: 1600
Total ulasan terkumpul: 1800
Total ulasan terkumpul: 2000
Total ulasan terkumpul: 2200
Total ulasan terkumpul: 2400
Total ulasan terkumpul: 2600
Total ulasan terkumpul: 2800
Total ulasan terkumpul: 3000
Total ulasan terkumpul: 3200
Total ulasan terkumpul: 3400
Total ulasan terkumpul: 3600
Total ulasan terkumpul: 3800
Total ulasan terkumpul: 4000
Total ulasan terkumpul: 4200
Total ulasan terkumpul: 4400
Total ulasan terkumpul: 4600
Total ulasan terkumpul: 4800
Total ulasan terkumpul: 5000
Total ulasan terkumpul: 5200
Total ulasan terkumpul: 5400
Total ulasan terkumpul: 5600
Total ulasan terkumpul: 5800
Total ulasan terkumpul: 6000
Total ulasan terkumpul: 6200
Total ulasan terkump

In [17]:
df = pd.read_csv(f'{file_name}')
df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,034db07e-4c41-4999-8933-b5b05a6bca3d,Candra Dwi Aprida,https://play-lh.googleusercontent.com/a/ACg8oc...,Semoga tetap responsif aplikasi nya,5,0,3.331.1,2025-09-28 21:37:14,NaN,NaN,3.331.1
1,c8e9f72a-be60-472d-be22-ffd9ba5a45ee,Sarifudin Mancung,https://play-lh.googleusercontent.com/a/ACg8oc...,Apk udah ga sebagus dulu. BYE TOKPED!,1,0,NaN,2025-09-28 20:17:36,NaN,NaN,NaN
2,c14cbaca-01f1-4069-b6fc-001d4274602d,Anton Christianto,https://play-lh.googleusercontent.com/a-/ALV-U...,tidak bisa menggunakan promo apapun sejak juli...,1,0,3.331.1,2025-09-28 20:05:32,NaN,NaN,3.331.1
3,7394c6c4-d922-4db8-b543-e716d8ec61c3,Rafael Riezkya Arnandra,https://play-lh.googleusercontent.com/a-/ALV-U...,"Tampilan desihn UI/UX nya bagus, tapi kadang b...",4,0,NaN,2025-09-28 17:14:39,NaN,NaN,NaN
4,d3d9ba6a-0a1e-4184-8fca-33533dc11b6d,Aldy Adela,https://play-lh.googleusercontent.com/a-/ALV-U...,nice,5,0,3.329.0,2025-09-28 15:51:18,NaN,NaN,3.329.0


In [11]:
df_batch = pd.read_csv('Tokopedia.csv')
df_batch.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,e59cbaf2-3536-4cbe-a60b-1abc7ff37b8a,Hendra,https://play-lh.googleusercontent.com/a-/ALV-U...,Banyak penjual barang yang menjual tidak sesua...,1,0,NaN,2024-02-03 10:16:34,"Hi Toppers, silakan sampaikan kritik/saran ata...",2024-02-03 10:20:07,NaN
1,2610c38c-b4fe-4039-a22b-39e1e0142561,Anang Ahmad Mulawarman Samalewa,https://play-lh.googleusercontent.com/a-/ALV-U...,Gk jelas tiba² pembatalan pesanan katanya tida...,1,0,3.251,2024-02-03 10:11:11,"Hi Toppers, silakan sampaikan kritik/saran ata...",2024-02-03 10:20:09,3.251
2,96df241d-e7e8-4581-8a7e-476365fa38b2,Mohamad Allwi,https://play-lh.googleusercontent.com/a-/ALV-U...,. Expedisi nya ga jelas.. Kirim paket aja smpa...,1,0,3.251,2024-02-03 10:09:59,"Toppers, mohon maaf atas kendala yang kamu ala...",2024-02-03 10:20:05,3.251
3,ab2222f9-b243-47be-9a08-cc799d7a8775,Queensha Amara,https://play-lh.googleusercontent.com/a/ACg8oc...,Parah beli barang malah di tuduh manipulasi da...,1,0,3.250,2024-02-03 10:09:07,"Halo Toppers, terima kasih atas rating yang ka...",2024-02-03 10:20:08,3.250
4,63e11d25-48c4-431d-9b66-4e57286cadc1,Sapuani Basri Vlog,https://play-lh.googleusercontent.com/a-/ALV-U...,terimakasih ok bangat,5,0,3.250,2024-02-03 10:05:37,"Toppers, terima kasih atas rating yang kamu be...",2024-02-03 10:20:04,3.250
